In [ ]:
import pandas as pd

In [ ]:
tokens_df = pd.read_csv("results/tokens.tsv", names=["study", "notation", "slug", "token"], delimiter="\t")

In [ ]:
tokens_df.query("study=='gapminder'").groupby("token").count().reset_index().sort_values("slug")

41